# Import

In [1]:
data_path='data/'
api_key='ehlVSIKSMiugJZHlii3sU7OKCexe4MIlnYenGric'
import pandas as pd 
import numpy as np

# Analysis

## Take  a look on dataset

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(f"{data_path}epi_r.csv")
for name, count in df.nunique().items():
    print(f"{name}: {count}")

title: 17736
rating: 8
calories: 1858
protein: 282
fat: 326
sodium: 2434
#cakeweek: 2
#wasteless: 2
22-minute meals: 2
3-ingredient recipes: 2
30 days of groceries: 2
advance prep required: 2
alabama: 2
alaska: 2
alcoholic: 2
almond: 2
amaretto: 2
anchovy: 2
anise: 2
anniversary: 2
anthony bourdain: 2
aperitif: 2
appetizer: 2
apple: 2
apple juice: 2
apricot: 2
arizona: 2
artichoke: 2
arugula: 2
asian pear: 2
asparagus: 2
aspen: 2
atlanta: 2
australia: 2
avocado: 2
back to school: 2
backyard bbq: 2
bacon: 2
bake: 2
banana: 2
barley: 2
basil: 2
bass: 2
bastille day: 2
bean: 2
beef: 2
beef rib: 2
beef shank: 2
beef tenderloin: 2
beer: 2
beet: 2
bell pepper: 2
berry: 2
beverly hills: 2
birthday: 2
biscuit: 2
bitters: 2
blackberry: 2
blender: 2
blue cheese: 2
blueberry: 2
boil: 2
bok choy: 2
bon appétit: 2
bon app��tit: 2
boston: 2
bourbon: 2
braise: 2
bran: 2
brandy: 2
bread: 2
breadcrumbs: 2
breakfast: 2
brie: 2
brine: 2
brisket: 2
broccoli: 2
broccoli rabe: 2
broil: 2
brooklyn: 2
brown r

The dataset have target feature:
```rating```.

There is also features: ```title, calories, protein, fat, sodium```



Dataset have 2 categorical features after the One-Hot Encoding conversion.

first categorical feature is:

Ingredients: ```almond, anchovy, apple, avocado, bacon```

second categorical feature is tags

The tags may contain:
* Recipe characteristics: ```#cakeweek, #wasteless, 22-minute meals, 3-ingredient recipes...```

* Geographical affiliation: ```alabama, arizona, boston, france, italy...```

* Dietary restrictions and features: ```dairy free, gluten free, vegan...```

* Cooking methods: ```bake, boil, grill, slow cooker...```

* Holidays and events: ```christmas, halloween, valentine's day...```

* Machinery and tools: ```blender, food processor, pressure cooker, microwave...```

## Let's sort columns in the dataset, and get nutrients for ingredients

### this is list of nutrients, what we need

In [3]:
nutrients_list = [
    # Основные макронутриенты 
    "Total lipid (fat)",
    "Fatty acids, total saturated",
    "Fatty acids, total monounsaturated",
    "Fatty acids, total polyunsaturated",
    "Fatty acids, total trans",
    "Cholesterol",
    "Carbohydrate, by difference",
    "Sodium, Na", 
    "Fiber, total dietary",
    "Protein",
    "Sugars, Total",  
    "Energy", 
    "Vitamin A, RAE",  
    "Vitamin C, total ascorbic acid", 
    "Vitamin D (D2 + D3)",
    "Vitamin E (alpha-tocopherol)",
    "Vitamin K (phylloquinone)",
    "Thiamin",
    "Riboflavin",
    "Niacin",
    "Vitamin B-6",
    "Folate, total",
    "Vitamin B-12",
    
    # Минералы
    "Calcium, Ca",
    "Iron, Fe",
    "Phosphorus, P",
    "Magnesium, Mg",
    "Zinc, Zn",
    "Copper, Cu",
    "Manganese, Mn",
    "Selenium, Se",
    "Potassium, K",
    
    # Специфичные компоненты 
    "Ash",
    "Nitrogen",
    "Water",
    
    # Разные формы сахаров
    "Fructose",
    "Glucose",
    "Sucrose",
    "Galactose"
]


### Funcs for primary sort ingredients and get cache for nutrients 

In [4]:
import requests
import json
def load_nutrients_cache(path_to_file):
    try:
        with open(path_to_file, 'r', encoding='utf-8') as f:
            try:
                return json.load(f)
            except:
                return {}
    except:
        return {}
def save_to_cache(path_to_file, ingredient, nutrient_dict, cache):
    cache[ingredient]=nutrient_dict
    with open(path_to_file, 'w', encoding='utf-8') as f:
        json.dump(cache, f, indent=4, ensure_ascii=False)

def get_nutrients_info(ingredient):
    ingredient_info=None
    params = {
        "query": f'"{ingredient}"',
        "api_key": api_key,
        "dataType": ["Foundation"]  
    }
    cache=load_nutrients_cache(f"{data_path}ingridient_nutr_cache.json")
    non_ingredient_cache=load_nutrients_cache(f"{data_path}not_ingridient_cache.json")
    if ingredient in cache:
        return cache[ingredient]
    elif ingredient not in non_ingredient_cache:
        response = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', params=params)
        if response.status_code == 200:
            ingredient_info = response.json().get('foods')
            if ingredient_info and ingredient_info[0].get('score') < 200:
                ingredient_info=None
            if ingredient_info:
                ingredient_info=ingredient_info[0].get('foodNutrients')
                nutrient_df=pd.json_normalize(ingredient_info)[['nutrientName','unitName','value']]
                nutrient_dict=nutrient_df[nutrient_df['nutrientName'].isin(nutrients_list)].to_dict()       
                save_to_cache(f"{data_path}ingridient_nutr_cache.json",ingredient, nutrient_dict, cache)
    if not ingredient_info:
        save_to_cache(f"{data_path}not_ingridient_cache.json",ingredient, ingredient_info, non_ingredient_cache)
    return ingredient_info
def is_ingredient(ingredient):
    data=get_nutrients_info(ingredient)
    return True if data else False


In [5]:
features =[]
for column in df.columns:
    for ingredients in column.split("/"):
        if is_ingredient(ingredients):
            features.append(column)
            break
print("a list of possible ingredients")
print (features)


a list of possible ingredients
['fat', 'sodium', 'almond', 'apple', 'apple juice', 'apricot', 'arugula', 'asparagus', 'avocado', 'bacon', 'banana', 'barley', 'bean', 'beef', 'beef tenderloin', 'beet', 'blackberry', 'blueberry', 'boil', 'bok choy', 'braise', 'bran', 'bread', 'breakfast', 'broccoli', 'broil', 'brussel sprout', 'bulgur', 'butter', 'buttermilk', 'cabbage', 'cantaloupe', 'carrot', 'cashew', 'cauliflower', 'celery', 'cheddar', 'cherry', 'chestnut', 'chicken', 'chickpea', 'cobbler/crumble', 'coconut', 'cod', 'cookie', 'cookies', 'corn', 'cottage cheese', 'crab', 'cranberry', 'cream cheese', 'cucumber', 'dill', 'egg', 'eggplant', 'fat free', 'feta', 'fig', 'fish', 'garlic', 'grains', 'grape', 'grapefruit', 'green onion/scallion', 'ham', 'hazelnut', 'hominy/cornmeal/masa', 'honeydew', 'hummus', 'kale', 'kiwi', 'kosher', 'lamb', 'leek', 'lentil', 'lettuce', 'low fat', 'low/no sugar', 'macadamia nut', 'mango', 'meat', 'melon', 'milk/cream', 'monterey jack', 'mozzarella', 'mushroo

In [6]:
non_ingreedients = []
for column in df.columns:
    for non_ingreedient in column.split("/"):
        if not is_ingredient(non_ingreedient):
            non_ingreedients.append(column)
            break
print("a list of possible non ingredients")
print (non_ingreedients)


a list of possible non ingredients
['title', 'rating', 'calories', 'protein', '#cakeweek', '#wasteless', '22-minute meals', '3-ingredient recipes', '30 days of groceries', 'advance prep required', 'alabama', 'alaska', 'alcoholic', 'amaretto', 'anchovy', 'anise', 'anniversary', 'anthony bourdain', 'aperitif', 'appetizer', 'arizona', 'artichoke', 'asian pear', 'aspen', 'atlanta', 'australia', 'back to school', 'backyard bbq', 'bake', 'basil', 'bass', 'bastille day', 'beef rib', 'beef shank', 'beer', 'bell pepper', 'berry', 'beverly hills', 'birthday', 'biscuit', 'bitters', 'blender', 'blue cheese', 'bon appétit', 'bon app��tit', 'boston', 'bourbon', 'brandy', 'breadcrumbs', 'brie', 'brine', 'brisket', 'broccoli rabe', 'brooklyn', 'brown rice', 'brownie', 'brunch', 'buffalo', 'buffet', 'bulgaria', 'burrito', 'butternut squash', 'butterscotch/caramel', 'cake', 'california', 'calvados', 'cambridge', 'campari', 'camping', 'canada', 'candy', 'candy thermometer', 'capers', 'caraway', 'cardamom

This API method doesn't work very well, so we'll sort the features manually.

remove non_ingredients from features

In [7]:
non_ingredients = [
    'fat',               
    'sodium',            
    'boil',              
    'braise',            
    'broil',            
    'breakfast',        
    'cobbler/crumble',  
    'cookie',           
    'cookies',        
    'fat free',        
    'kosher',      
    'low fat',          
    'low/no sugar',    
    'no sugar added',   
    'no-cook',     
    'raw',             
    'stuffing/dressing',
    'summer',       
    'winter',         
    'wheat/gluten-free'
]
for value in non_ingredients:
    if value in features:
        if value not in df.columns:
            print(f"problem with: {value}")
        features.remove(value)

add actually ingredients from dataset to features

In [8]:
actually_ingredients=actually_ingredients = [
    'anchovy', 'anise', 'apple juice', 'artichoke',
    'asian pear', 'basil', 'bass', 'beef rib',
    'beef shank', 'beer', 'bell pepper', 'berry',
    'bitters', 'blue cheese', 'brie', 'brisket',
    'broccoli rabe', 'brown rice', 'brownie', 'butternut squash',
    'butterscotch/caramel', 'cake', 'calvados', 'capers',
    'caraway', 'cardamom', 'caviar', 'chard',
    'cheese', 'chile', 'chile pepper', 'chili',
    'chive', 'chocolate', 'cilantro', 'cinnamon',
    'citrus', 'clam', 'clove', 'coffee',
    'collard greens', 'cornmeal', 'couscous', 'cranberry sauce',
    'crêpe', 'currant', 'curry', 'custard',
    'date', 'dried fruit', 'duck', 'egg nog',
    'endive', 'escarole', 'fennel', 'flat bread',
    'fontina', 'fruit juice', 'ginger', 'goat cheese',
    'goose', 'gouda', 'granola', 'green bean',
    'ground beef', 'ground lamb', 'guava', 'halibut',
    'hamburger', 'herb', 'hominy/cornmeal/masa', 'honey',
    'horseradish', 'ice cream', 'iced coffee', 'iced tea',
    'jalapeño', 'jam or jelly', 'jerusalem artichoke', 'jícama',
    'kumquat', 'lamb chop', 'lamb shank', 'lasagna',
    'leafy green', 'legume', 'lemon', 'lemon juice',
    'lemongrass', 'lima bean', 'lime', 'lime juice',
    'lingonberry', 'lobster', 'macaroni and cheese', 'maple syrup',
    'mayonnaise', 'meatball', 'meatloaf', 'milk/cream',
    'mint', 'molasses', 'muffin', 'mussel',
    'mustard greens', 'noodle', 'nutmeg', 'octopus',
    'okra', 'omelet', 'oregano', 'orzo',
    'papaya', 'paprika', 'parsley', 'parsnip',
    'passion fruit', 'persimmon', 'phyllo/puff pastry dough', 'poblano',
    'pomegranate juice', 'poppy', 'pork rib', 'pork tenderloin',
    'pot pie', 'potato salad', 'prosciutto', 'quail',
    'quiche', 'quince', 'rabbit', 'rack of lamb',
    'radicchio', 'radish', 'rhubarb', 'rosemary',
    'rosé', 'rum', 'saffron', 'sage',
    'sake', 'salad dressing', 'sardine', 'scallop',
    'scotch', 'sesame oil', 'shellfish', 'sherry',
    'snapper', 'sorbet', 'soufflé/meringue', 'soup/stew',
    'sour cream', 'sourdough', 'soy sauce', 'spice',
    'squid', 'stock', 'stuffing/dressing', 'sugar snap pea',
    'sweet potato/yam', 'swiss cheese', 'swordfish', 'tamarind',
    'tangerine', 'tapioca', 'tarragon', 'tea',
    'tofu', 'tortillas', 'tree nut', 'triple sec',
    'tropical fruit', 'trout', 'turnip', 'vanilla',
    'veal', 'venison', 'vermouth', 'vinegar',
    'waffle', 'wasabi', 'watercress', 'watermelon',
    'white wine', 'wine', 'yellow squash', 'yuca'
]
for value in actually_ingredients:
    if value not in features:
        if value not in df.columns:
            print(f"problem with: {value}")
        features.append(value)


## lets sort columns in our dataset


In [9]:
X=df[features]
y=df['rating']

In [10]:
print(X.shape) 
print(y.shape) 

X.sum().sort_values(ascending=False)

(20052, 322)
(20052,)


onion         2238.0
tomato        2140.0
milk/cream    1995.0
egg           1768.0
herb          1681.0
               ...  
caviar           1.0
waffle           1.0
crêpe            1.0
quiche           1.0
sorbet           1.0
Length: 322, dtype: float64

## Let's start tuning the hyperparameters

это чисто препроцессинг, то что я сделал выше мб оформлю его так же шобы в пайплайн запихать 

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

#### Train-test-validation split 

In [12]:
def rating_to_сat(y):
    bins = [0, 2, 4, 6]
    labels = ['bad', 'ok', 'good']
    return pd.cut(y, bins=bins, labels=labels)

class TrainValidationTest(BaseEstimator, TransformerMixin):
        def __init__(self, test_size=0.2, random_state=21, stratified=True):
            self.test_size = test_size
            self.random_state = random_state
            self.stratified = stratified
        def split(self, X , y):
            stratify = y if self.stratified else None
            X_train, X_test, y_train, y_test =train_test_split(
                X, y, test_size=self.test_size, random_state=self.random_state, stratify=stratify
            )
            X_train, X_valid, y_train, y_valid =train_test_split(
                X_train, y_train, test_size=self.test_size/(1-self.test_size), random_state=self.random_state, stratify=stratify
            )
            return X_train, X_valid, X_test, y_train, y_valid, y_test

#### Model selection class


In [ ]:
from copy import deepcopy
from itertools import product
from sklearn.metrics import get_scorer
def manual_grid_search(X_train, y_train, X_valid, y_valid, model_estimator,param_grid, cv, scoring='accuracy_score',  n_jobs=-1, error_score=np.nan):
    res=[]
    estimator=deepcopy(model_estimator)
    total_combinations = np.prod([len(v) for v in param_grid.values()])
    # get scorer for valid score
    scorer = get_scorer(scoring)

    for params in tqdm(list(product(*param_grid.values())), total=total_combinations, desc="GridSearch"):

        current_model = deepcopy(estimator)
        current_model.set_params(**(dict(zip(param_grid.keys(), params))))
        try:
            score=cross_val_score(
                current_model, X_train, y_train, cv=cv, scoring=scoring, n_jobs=n_jobs,error_score=error_score
            )

            current_model.fit(X_train, y_train)
            valid_score=scorer(current_model, X_valid, y_valid)

            res.append({
                'train_score': score.mean(),
                'valid_score': valid_score,
                'params': dict(zip(param_grid.keys(), params))
            })


        except Exception as e:
            print(f"Error with params {params}: {str(e)}")
            res.append({
                **(dict(zip(param_grid.keys(), params))),
                'train_score': 'Na',
                'valid_score': 'Na'
            })
            continue
        
    return pd.DataFrame(res).sort_values('valid_score', ascending=False)

class ModelSelection:
    def __init__ (self,estimators,estimators_name, grid_dict, scoring='accuracy_score'):
        self.estimators=estimators
        self.scoring=scoring
        self.estimators_name=estimators_name
        self.grid_dict=grid_dict
        self.results=pd.DataFrame()
        self.best_res=[]
    def choose(self,X_train, y_train, X_valid, y_valid, cv=2):
        """
        Method choose() takes X_train, y_train, X_valid, y_valid, optional cv, 
        and returns the name of the best model among all the models on the validation set
        """
        for estimator in self.estimators:
            model_name= self.estimators_name[self.estimators.index(estimator)]
            search_res=manual_grid_search(X_train,y_train,X_valid, y_valid,estimator,self.grid_dict[model_name], cv=cv, scoring=self.scoring)
            search_res['model']=model_name
            best_params = search_res.head(1)['params'].item()
            best_train_score = search_res.head(1)['train_score'].item()
            best_valid_score = search_res.head(1)['valid_score'].item()
            self.results=pd.concat([self.results, search_res], axis=0, ignore_index=True)
            self.best_res.append({
                'model': model_name,
                'params': best_params,
                'valid_score': best_valid_score,
                'train_score': best_train_score
            })
            
            print(f"Best params for model{model_name}: {best_params}")
            print(f"training accuracy: {best_train_score:.3f}")
            print(f"Best on validation set accuracy score: {best_valid_score:.3f}")
        self.best_res=pd.DataFrame(self.best_res)
        self.best_res.sort_values('valid_score', ascending=False,inplace=True)
        best_model_name=self.best_res.head(1)['model'].item()
        print(f"\nBest model: {best_model_name}\nwith params: {self.best_res.head(1)['params'].item()}\nand score: {self.best_res.head(1)['valid_score'].item()}")
        return self.results.sort_values('valid_score', ascending=False)
    # def show_results(self):
        # return self.results.groupby(by="model").sort_values('valid_score', ascending=False)
    def fit_score(self,X_comb,y_comb, X_test, y_test):
        params=self.best_res.head(1)['params'].item()
        model_name=self.best_res.head(1)['model'].item()
        print(f'Taking model: {model_name} with params:{params}')
        best_estimator=self.estimators[self.estimators_name.index(model_name)]
        best_estimator.set_params(**(params))
        best_estimator.fit(X_comb,y_comb)
        scorer=get_scorer(self.scoring)
        print("the final score is :",(scorer(best_estimator,X_test,y_test)))        

In [15]:
def fit_score(estimator, params, X_comb, y_comb, X_test, y_test, scoring='accuracy_score'):
        estimator.set_params(**(params))
        estimator.fit(X_comb,y_comb)
        scorer=get_scorer(scoring)
        print("the final score is :",(scorer(estimator,X_test,y_test)))  

# Regression serch params

In [16]:
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge

estimators = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    RandomForestRegressor(random_state=21, n_jobs=-1),
    GradientBoostingRegressor(random_state=21),
    XGBRegressor(random_state=21, n_jobs=-1),
    KernelRidge(),
]

estimators_name = [
    'LinearRegression',
    'Ridge',
    'Lasso',
    'ElasticNet',
    'RandomForest',
    'GradientBoosting',
    'XGBoost',
    'KernelRidge',
]

grid_dict = {
    'LinearRegression': {
        'fit_intercept': [True, False],
        'positive': [False]
    },
    'Ridge': {
        'alpha': [0.1, 1.0, 10.0],  
        'solver': ['svd', 'cholesky', 'saga']  
    },
    'Lasso': {
        'alpha': [0.01, 0.1, 1.0],  
        'selection': ['cyclic']
    },
    'ElasticNet': {
        'alpha': [0.01, 0.1, 1.0],
        'l1_ratio': [0.3, 0.5, 0.7] 
    },
    'RandomForest': {
        'n_estimators': [50,100, 200], 
        'max_depth': [15,30, None],  
        'min_samples_split': [2, 5,10],
        'max_features': ['sqrt']
    },
    'GradientBoosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5],
        'subsample': [0.8, 1.0]
    },
    'XGBoost': {
        'n_estimators': [100, 200],
        'max_depth': [3, 6],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    },
    'KernelRidge': {
        'alpha': [0.1, 1.0, 10.0],
        'kernel': ['rbf'],
        'gamma': [None, 0.1]
    }
}

Regression grid search


In [17]:
scoring='neg_root_mean_squared_error'


splitter = TrainValidationTest(stratified=False)
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split(X, y)
search_parametrs= ModelSelection(estimators,estimators_name, grid_dict, scoring)
result_serching=search_parametrs.choose(X_train, y_train, X_valid, y_valid)
X_comb=pd.concat([X_train, X_valid], axis=0, ignore_index=True)
y_comb=pd.concat([y_train, y_valid], axis=0, ignore_index=True)


GridSearch:   0%|          | 0/2 [00:00<?, ?it/s]

Best params for modelLinearRegression: {'fit_intercept': True, 'positive': False}
training accuracy: -1.339
Best on validation set accuracy score: -1.292


GridSearch:   0%|          | 0/9 [00:00<?, ?it/s]

Best params for modelRidge: {'alpha': 10.0, 'solver': 'saga'}
training accuracy: -1.320
Best on validation set accuracy score: -1.282


GridSearch:   0%|          | 0/3 [00:00<?, ?it/s]

...

Best params for modelGradientBoosting: {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 5, 'subsample': 1.0}
training accuracy: -1.332
Best on validation set accuracy score: -1.276


GridSearch:   0%|          | 0/32 [00:00<?, ?it/s]

Best params for modelXGBoost: {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'colsample_bytree': 1.0}
training accuracy: -1.330
Best on validation set accuracy score: -1.273


GridSearch:   0%|          | 0/6 [00:00<?, ?it/s]

Best params for modelKernelRidge: {'alpha': 1.0, 'kernel': 'rbf', 'gamma': 0.1}
training accuracy: -1.313
Best on validation set accuracy score: -1.275

Best model: RandomForest
with params: {'n_estimators': 200, 'max_depth': 30, 'min_samples_split': 2, 'max_features': 'sqrt'}
and score: -1.271343441880819


In [18]:
print("Calc score on the best model")

best_model_name = result_serching.iloc[0]['model']
best_estimator = estimators[estimators_name.index(best_model_name)]
params= result_serching.iloc[0]['params']
print(f'Taking model: {best_model_name} with params:{params}')
fit_score(best_estimator, params, X_comb,y_comb, X_test,y_test, scoring=scoring)

Calc score on the best model
Taking model: RandomForest with params:{'n_estimators': 200, 'max_depth': 30, 'min_samples_split': 2, 'max_features': 'sqrt'}
the final score is : -1.2607774800247062


let's analyze the results

In [19]:
result_serching.head()

,train_score,valid_score,params,model
23,-1.317030,-1.271343,"{'n_estimators': 200, 'max_depth': 30, 'min_sa...",RandomForest
24,-1.317769,-1.271758,"{'n_estimators': 100, 'max_depth': 30, 'min_sa...",RandomForest
66,-1.329763,-1.273091,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",XGBoost
67,-1.327541,-1.273299,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",XGBoost
25,-1.319519,-1.273352,"{'n_estimators': 50, 'max_depth': 30, 'min_sam...",RandomForest


In [20]:
model_name = result_serching.iloc[2]['model']
estimator = estimators[estimators_name.index(model_name)]
params= result_serching.iloc[2]['params']
print(f'Taking model: {model_name} with params:{params}')
fit_score(estimator, params, X_comb, y_comb, X_test, y_test, scoring=scoring)


Taking model: XGBoost with params:{'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'colsample_bytree': 1.0}
the final score is : -1.2665302481544887


Lets take  RandomForest with params:{'n_estimators': 200, 'max_depth': 30, 'min_samples_split': 2, 'max_features': 'sqrt'}
with the final score : -1.260777480024706

In [21]:
RandomForestRegressor_model=RandomForestRegressor(random_state=21, n_jobs=-1)
RandomForestRegressor_model.set_params(**(result_serching.iloc[0]['params']))


RandomForestRegressor(max_depth=30, max_features='sqrt', n_estimators=200,
                      n_jobs=-1, random_state=21)

lets try do model that classifays on regression purpose 

In [25]:
def rating_to_сat(y):
    bins = [0, 2, 4, 6]
    labels = ['bad', 'so-so', 'great']
    return pd.cut(y, bins=bins, labels=labels)
class RegressionToClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, regressor=None, bins=[0, 2, 4, 6], labels=['bad', 'so-so', 'great']):
        self.regressor = regressor if regressor is not None else RandomForestRegressor()
        self.bins = bins
        self.labels = labels
    
    def fit(self, X, y):
        self.regressor.fit(X, y)
        return self
    
    def predict(self, X):
        y_pred = self.regressor.predict(X)
        return pd.cut(y_pred, bins=self.bins, labels=self.labels, include_lowest=True)
    
    def predict_proba(self, X):
        raise NotImplementedError("predict_proba is not implemented for this classifier")
regresRegressionToClassifier = RegressionToClassifier(RandomForestRegressor_model)
regresRegressionToClassifier.fit(X_comb,y_comb)
y_test_classification=rating_to_сat(y_test)
y_pred=regresRegressionToClassifier.predict(X_test)
accuracy_score(y_test_classification,y_pred)


TypeError: '<' not supported between instances of 'float' and 'str'

# classification

## classes rounded to the closest integer

prepare data

In [ ]:
def closest_int_class(y):
    return y.round().astype(int)
y_closest_int=closest_int_class(y)

scoring='accuracy_score'


splitter = TrainValidationTest(stratified=False)
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split(X, y_closest_int)
X_comb=pd.concat([X_train, X_valid], axis=0, ignore_index=True)
y_comb=pd.concat([y_train, y_valid], axis=0, ignore_index=True)


diffrent models variants

search best model

In [ ]:
search_parametrs= ModelSelection(estimators,estimators_name, grid_dict, scoring)
result_serching=search_parametrs.choose(X_train, y_train, X_valid, y_valid)